### SXDG Case

In [1]:
import qiskit

from qiskit_aer import AerSimulator

from rivet_transpiler import transpile
from rivet_transpiler import get_full_map
from rivet_transpiler import get_sinusoids
from rivet_transpiler import get_litmus_circuit

from qiskit_ibm_runtime.fake_provider import fake_backend

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_ibm_runtime.fake_provider import FakeGuadalupeV2
from qiskit_ibm_runtime.fake_provider import FakeMontrealV2

In [2]:
# Parameters

OPTIMIZATION_LEVEL = 3

SHOTS_COUNT = 1000
SEED_TRANSPILER = 1234

LAYOUT_METHOD = None
ROUTING_METHOD = None

In [3]:
QUBITS_COUNT = 5

litmus_circuit = get_litmus_circuit(QUBITS_COUNT, "Litmus")

litmus_circuit.draw()

┌──────────────┐                    ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(Litmus_0) ├──■─────────────────┤ X ├───░────
            ├──────────────┤┌─┴─┐               └─┬─┘   ░    
Litmus_0_1: ┤ Rz(Litmus_1) ├┤ X ├──■──────────────┼─────░────
            ├──────────────┤└───┘┌─┴─┐            │     ░    
Litmus_0_2: ┤ Rz(Litmus_2) ├─────┤ X ├──■─────────┼─────░────
            ├──────────────┤     └───┘┌─┴─┐       │     ░    
Litmus_0_3: ┤ Rz(Litmus_3) ├──────────┤ X ├──■────┼─────░────
            ├──────────────┤          └───┘┌─┴─┐  │     ░    
Litmus_0_4: ┤ Rz(Litmus_4) ├───────────────┤ X ├──■─────░────
            └──────────────┘               └───┘        ░

In [4]:
fake_backend = FakeMontrealV2()
# fake_backend = FakeGuadalupeV2()
# fake_backend = FakeLimaV2()


backend = AerSimulator.from_backend(fake_backend)

backend

AerSimulator('aer_simulator_from(fake_montreal)'
             noise_model=<NoiseModel on ['x', 'cx', 'measure', 'id', 'sx', 'reset']>)

In [5]:
backend.options.noise_model = None

### Transpile Left

In [6]:
def transpile_left(central_circuit, left_circuit,
                   backend=None, **key_arguments):

    # Left Initial Layout

    if central_circuit.layout is None:

        left_initial_layout = list(range(left_circuit.num_qubits))

    else:

        initial_layout = central_circuit.layout.initial_layout
        input_qubit_mapping = central_circuit.layout.input_qubit_mapping

        initial_map = [initial_layout[qubit] for qubit in input_qubit_mapping]

        left_initial_layout = initial_map[:left_circuit.num_qubits]

    # Transpile and Compose

    key_arguments['initial_layout'] = left_initial_layout

    inverted_left_circuit = left_circuit.inverse()

    transpiled_inverted_left_circuit = transpile(
        inverted_left_circuit,
        backend,
        **key_arguments)

    transpiled_left_circuit = transpiled_inverted_left_circuit.inverse()

    transpiled_left_circuit._layout = transpiled_inverted_left_circuit.layout

    resulting_circuit = central_circuit.compose(transpiled_left_circuit,
                                                front=True)

    # No Layout

    if transpiled_left_circuit.layout is None:

        return resulting_circuit

    # Left Routing

    if transpiled_left_circuit.layout.final_layout is None:

        left_routing = list(range(transpiled_left_circuit.num_qubits))

    else:
        left_routing = [transpiled_left_circuit.layout.final_layout[qubit]
                        for qubit in transpiled_left_circuit.qubits]

    # Central Routing

    if (central_circuit.layout is None or
            central_circuit.layout.final_layout is None):

        central_routing = list(range(central_circuit.num_qubits))

    else:
        central_routing = [central_circuit.layout.final_layout[qubit]
                           for qubit in central_circuit.qubits]

    # Final Routing

    final_routing = [central_routing[qubit] for qubit in left_routing]

    # Final Layout

    final_layout = qiskit.transpiler.Layout.from_intlist(final_routing, *resulting_circuit.qregs)

    # Initial Layout

    input_qubit_mapping = transpiled_left_circuit.layout.input_qubit_mapping

    initial_map = get_full_map(transpiled_left_circuit)

    initial_layout = transpiled_left_circuit.layout.initial_layout.copy()

    for virtual, physical in zip(input_qubit_mapping, initial_map):

        initial_layout[virtual] = physical

    # Transpile Layout

    transpile_layout = qiskit.transpiler.TranspileLayout(
        input_qubit_mapping=input_qubit_mapping,
        initial_layout=initial_layout,
        final_layout=final_layout
    )

    resulting_circuit._layout = transpile_layout

    # Printouts

    # print("left_routing:", left_routing)
    # print("central_routing:", central_routing)
    # print("final_routing:", final_routing)
    # print("final_layout:", final_layout)

    return resulting_circuit

### Run

In [7]:
QUBITS_COUNT = litmus_circuit.num_qubits

qubits = list(range(QUBITS_COUNT))

# Measurement Part

measurement_circuit = qiskit.QuantumCircuit(QUBITS_COUNT, QUBITS_COUNT)
measurement_circuit.measure(qubits, qubits)

# Transpile Left

transpiled_measurement_circuit = transpile(
    measurement_circuit,
    backend,
    layout_method=LAYOUT_METHOD,
    routing_method=ROUTING_METHOD,
    seed_transpiler=SEED_TRANSPILER,
    optimization_level=OPTIMIZATION_LEVEL
)

In [27]:
key_arguments = {
    'layout_method': LAYOUT_METHOD,
    'routing_method': ROUTING_METHOD,
    'seed_transpiler': SEED_TRANSPILER,
    'optimization_level': OPTIMIZATION_LEVEL
}

central_circuit = transpiled_measurement_circuit
left_circuit = litmus_circuit

# Left Initial Layout

if central_circuit.layout is None:

    left_initial_layout = list(range(left_circuit.num_qubits))

else:

    initial_layout = central_circuit.layout.initial_layout
    input_qubit_mapping = central_circuit.layout.input_qubit_mapping

    initial_map = [initial_layout[qubit] for qubit in input_qubit_mapping]

    left_initial_layout = initial_map[:left_circuit.num_qubits]

# Transpile and Compose

key_arguments['initial_layout'] = left_initial_layout

inverted_left_circuit = left_circuit.inverse()

transpiled_inverted_left_circuit = transpile(
    inverted_left_circuit,
    backend,
    **key_arguments
    # layout_method=LAYOUT_METHOD,
    # routing_method=ROUTING_METHOD,
    # seed_transpiler=SEED_TRANSPILER,
    # optimization_level=OPTIMIZATION_LEVEL
)

In [28]:
transpiled_inverted_left_circuit.draw(fold=-1, idle_wires=False)

░             ┌───┐                                                                                                                                                                                                                                                                                                                                                                   
 Litmus_0_3 -> 4 ─░──────■──────┤ X ├─────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                  ░    ┌─┴─┐    └─┬─┘   ┌─┴─┐        ┌───┐                                                                                                                                                                                                                                                                                                                                              
  ancilla_6 -> 7 ──────┤ X ├──────■─────┤ X ├─────■──┤ X ├──■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                       └───┘            └───┘   ┌─┴─┐└─┬─┘┌─┴─┐     ┌───┐                                                                                                                                                                                                                                                                                                                               
 ancilla_9 -> 10 ───────────────────────────────┤ X ├──■──┤ X ├──■──┤ X ├──■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                └───┘     └───┘┌─┴─┐└─┬─┘┌─┴─┐     ┌───┐                                                                                                                                                                                           ┌───┐                                                                                                                
ancilla_11 -> 12 ──────────────────────────────────────────────┤ X ├──■──┤ X ├──■──┤ X ├──■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────────────────────■──┤ X ├──■─────────────────────────────────────────────────────────────────────────────────────────────────────────────
                  ░             ┌───┐                          └───┘     └───┘┌─┴─┐└─┬─┘┌─┴─┐                                                                                                                                               ┌─┴─┐        ┌───────────────────┐┌─┴─┐└─┬─┘┌─┴─┐     ┌───┐                                                                                                 
Litmus_0_0 -> 13 ─░──────■──────┤ X ├─────■───────────────────────────────────┤ X ├──■──┤ X ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────┤ X ├────────┤ Rz(-1.0*Litmus_3) ├┤ X ├──■──┤ X ├──■──┤ X ├──■──────────────────────────────────────────────────────────────────────────────────────────────
                  ░    ┌─┴─┐    └─┬─┘   ┌─┴─┐        ┌───┐                    └───┘     

In [20]:
# # Assign Parameters

# for index, parameter in enumerate(transpiled_left_circuit.parameters):

#     transpiled_left_circuit.assign_parameters({parameter: index}, inplace=True)

# # Run

# job = backend.run(transpiled_left_circuit, shots=SHOTS_COUNT)

# counts = job.result().get_counts()

# counts